## Baseball Savant pitchlog puller

This program pulls the pitch information from a mlb approved advanced statistics website.

In [5]:
#bring in all the libraries I'll need

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


import time
import string
import requests
from bs4 import BeautifulSoup
import re


import unidecode
import pandas as pd

# set driver options and import urls of each page of pitch lists
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')

pitch=pd.read_csv('E://HOU18/Playoffs 2017/2017playoff_games.csv',usecols=['pitch_url'])

pitch_url=[]
for i,x in pitch.iterrows():
    pitch_url.append(x[0])


# game number, count starts at 0
j=10
while j < 11:
    print(j)
    #initiate dataframe
    df = pd.DataFrame( columns = ['astro_batter', 'pitcher','pitch_type','batter','game_pitch',
                                       'pitch_count','plate_appearence','inning','result',
                                      'pitch_velo_(mph)','exit_velo_(mph)','launch_angle_(degrees)','distance_(ft)','xBA'])
    #initiate row maker
    row={}

    #initialize driver
   
    driver = webdriver.Chrome(ChromeDriverManager().install())

    time.sleep(4)
    driver.implicitly_wait(30)

    #go to baseball savant webpage
    driver.get(f'{pitch_url[j]}')
    time.sleep(4)

    #find number of rows will need to be pulled
    count=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[1]/td[7]/span')
   
    count=BeautifulSoup(count.text,'html.parser')

    #row counter
    n=1

    #loop through each row of the pitch log
    while n<=int(str(count)):
     
        #pull the info I want and parse it

        #batter's team, is it an Astro or the other team
        astro=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[5]/img')
        soup=BeautifulSoup(astro.get_attribute('src'),'html.parser')
        team=re.findall('\d+', str(soup))[0]
        if int(team) ==117:
            row['astro_batter']=1
        else:
            row['astro_batter']=0

        #Pitcher
        pitcher= driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[3]/span')
        soup=BeautifulSoup(pitcher.text,'html.parser')
        row['pitcher']=str(soup)

        #Pitch Type
        pitchtype=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[4]/span')
        soup=BeautifulSoup(pitchtype.text,'html.parser')
        row['pitch_type']=str(soup)

        # Batter
        batter=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[6]/span')
        soup=BeautifulSoup(batter.text,'html.parser')
        row['batter']=str(soup)

        #Game pitch number
        gamepitch=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[7]/span')
        soup=BeautifulSoup(gamepitch.text,'html.parser')
        row['game_pitch']=int(str(soup))

        # pitch number
        pitch=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[8]/span')
        soup=BeautifulSoup(pitch.text,'html.parser')
        row['pitch_count']=int(str(soup))

        #plate appearence
        plateappearence=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[9]/span')
        soup=BeautifulSoup(plateappearence.text,'html.parser')
        row['plate_appearence']=int(str(soup))

        #inning
        inning=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[10]/span')
        soup=BeautifulSoup(inning.text,'html.parser')
        row['inning']=int(str(soup))

        #result
        result=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[11]/span')
        soup=BeautifulSoup(result.text,'html.parser')
        row['result']=str(soup)

        #Velo
        velo=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[12]/span')
        soup=BeautifulSoup(velo.text,'html.parser')
        row['pitch_velo_(mph)']=str(soup)

        #exit
        exit=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[13]/span')
        soup=BeautifulSoup(exit.text,'html.parser')
        row['exit_velo_(mph)']=str(soup)

        #Launch
        launch=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[14]/span')
        soup=BeautifulSoup(launch.text,'html.parser')
        row['launch_angle_(degrees)']=str(soup)

        #distance
        dist=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[15]/span')
        soup=BeautifulSoup(dist.text,'html.parser')
        row['distance_(ft)']=str(soup)

        #xBA
        xba=driver.find_element_by_xpath(f'/html/body/div[3]/div[6]/table/tbody/tr[{n}]/td[16]/span')
        soup=BeautifulSoup(xba.text,'html.parser')
        row['xBA']=str(soup)

        #commit row to dataframe, clear row to start again
        df=df.append(row, ignore_index=True)
        row={}

        #increment
        n+=1

    time.sleep(4)

    #close webdriver
    driver.quit()

    #export collected info
    df.sort_values(by=['game_pitch'],inplace=True)
    df.set_index('game_pitch',inplace=True)
    df.to_csv(f'E://HOU18/Playoffs 2017/games/game {j+1}/savant log.csv')
    
    j+=1
    
print('done!')

driver.quit()

[WDM] - Cache is valid for [23/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\16142\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


10
 


C:\Users\16142\Anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://www.mlbstatic.com/team-logos/147.svg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\16142\Anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://www.mlbstatic.com/team-logos/117.svg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


done!


In [26]:
batter=[]
pitcher=[]
j=1
df=pd.read_csv(f'E://HOU18/Playoffs 2017/games/game {j+1}/savant log.csv')

for i,row in df.iterrows():
    x=row[2].split()
    pitcher.append(x[1])
    x=row[4].split()
    batter.append(x[1])

df['pitcher']=pitcher
df['batter']=batter

df.to_csv(f'E://HOU18/Playoffs 2017/games/game {j+1}/savant log.csv')

In [14]:
df.head()

,game_pitch,astro_batter,pitcher,pitch_type,batter,pitch_count,plate_appearence,inning,result,pitch_velo_(mph),exit_velo_(mph),launch_angle_(degrees),distance_(ft),xBA
0,1,0,Verlander,Four-Seam Fastball,Bogaerts,1,1,1,Called Strike,94.9,NaN,NaN,NaN,NaN
1,2,0,Verlander,Four-Seam Fastball,Bogaerts,2,1,1,Ball,96.2,NaN,NaN,NaN,NaN
2,3,0,Verlander,Four-Seam Fastball,Bogaerts,3,1,1,"In play, out(s)",96.5,84.8,75.0,140.0,0.00
3,4,0,Verlander,Four-Seam Fastball,Nunez,4,2,1,"In play, out(s)",96.2,76.0,-41.0,85.0,0.06
4,5,0,Verlander,Four-Seam Fastball,Benintendi,5,3,1,Called Strike,94.8,NaN,NaN,NaN,NaN
